<a href="https://colab.research.google.com/github/MuellerLeonard/text-preprocessing/blob/main/text_preprocessing_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

High RAM needed

In [1]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


Clone git



In [2]:
!git clone https://github.com/MuellerLeonard/text-preprocessing.git

fatal: destination path 'text-preprocessing' already exists and is not an empty directory.


In [3]:
!ls text-preprocessing/

processed_text_df_created.csv  text_corpus_clean_created.txt
processed_text_df.csv	       text_corpus_clean.txt
README.md		       text_corpus_created.txt
showcase.csv


Imports

In [4]:
import pandas as pd

Import the Dataset

In [5]:
df = pd.read_csv("/content/text-preprocessing/showcase.csv", nrows=100)
df.columns.values[0] = 'Date'

In [6]:
# test if DataFrame is correct

dri_df = df[df['Ticker'] == 'DRI']
print(dri_df)

Empty DataFrame
Columns: [Date, Unnamed: 0, Adj Close, item 5, item 4, item 5.02, item_1, item_1a, item 7, item 8.01, part_1_item_2, part_2_item_1a, net income, item 9.01, Ticker, Time, Filing, 10, Stock prices before (np.ndarray), Stock prices after (np.ndarray), Stock movement 1, Stock movement 7, Stock movement 30, 1 prior Adj Close, 1 prior Stock movement 1, 1 prior net income, 1 prior total stockholders’ equity, 1 prior retained earnings, 1 prior total operating expenses, 1 prior interest expense, 1 prior research and development, 1 prior net worth, 1 prior total assets, 1 prior inventories, 1 prior long-term debt, 1 prior short-term debt, 1 prior total net sales, 1 prior total liabilities and stockholders’ equity, 1 prior other income (expense), net, 2 prior Adj Close, 2 prior Stock movement 1, 2 prior net income, 2 prior total stockholders’ equity, 2 prior retained earnings, 2 prior total operating expenses, 2 prior interest expense, 2 prior research and development, 2 prior net

# Drop and Arrange the Dataset



In [7]:
import re
import numpy as np

## Drop Cells that are not needed

In [8]:
columns = df.columns

print(columns)

cleaned_columns = []

to_drop = ['item_1', 'item_1a', 'part_1_item_2', 'part_2_item_1a', 'item 1', 'item 2', 'item 3', 'item 4', 'item 5', 'item 6', 'item 7', 'item 8', 'item 9', 'item 12', 'item 601', 'item 8.1']
loop_drop = ['1 prior', '2 prior', '3 prior', '4 prior', '5 prior', '6 prior']
dropped = []

for dropit in to_drop:
  for drop in loop_drop:
    dropped.append(drop + ' ' + dropit)

print("DROPPED: ", dropped)

cleaned_columns = [col for col in columns if col not in dropped]

Index(['Date', 'Unnamed: 0', 'Adj Close', 'item 5', 'item 4', 'item 5.02',
       'item_1', 'item_1a', 'item 7', 'item 8.01',
       ...
       'item 1.04', 'item 1.05', 'item 5.08', 'item 5.05', 'item 601',
       'item 8.1', 'item 4.02', 'item 2', 'item 4.01', 'item 3.03'],
      dtype='object', length=164)
DROPPED:  ['1 prior item_1', '2 prior item_1', '3 prior item_1', '4 prior item_1', '5 prior item_1', '6 prior item_1', '1 prior item_1a', '2 prior item_1a', '3 prior item_1a', '4 prior item_1a', '5 prior item_1a', '6 prior item_1a', '1 prior part_1_item_2', '2 prior part_1_item_2', '3 prior part_1_item_2', '4 prior part_1_item_2', '5 prior part_1_item_2', '6 prior part_1_item_2', '1 prior part_2_item_1a', '2 prior part_2_item_1a', '3 prior part_2_item_1a', '4 prior part_2_item_1a', '5 prior part_2_item_1a', '6 prior part_2_item_1a', '1 prior item 1', '2 prior item 1', '3 prior item 1', '4 prior item 1', '5 prior item 1', '6 prior item 1', '1 prior item 2', '2 prior item 2', '3 pri

## Drop Rows that can not be experimented with

In [9]:
# drop rows with NaN inside a label column

df = df[(df["Adj Close"].notna()) & (df["Stock movement 1"].notna())]

df.reset_index(drop=True, inplace=True)

print(df.head(1))

   Date  Unnamed: 0  Adj Close item 5 item 4 item 5.02 item_1 item_1a item 7  \
0    95  2019-03-05  76.549049    NaN    NaN       NaN    NaN     NaN    NaN   

  item 8.01  ... item 1.04 item 1.05  item 5.08 item 5.05 item 601 item 8.1  \
0       NaN  ...       NaN       NaN        NaN       NaN      NaN      NaN   

   item 4.02 item 2 item 4.01 item 3.03  
0        NaN    NaN       NaN       NaN  

[1 rows x 164 columns]


## Add cells that might be interesting

### Adjusted Close Price of the next day

In [10]:
def get_price_next_day(row):
  """
  add next day price column to DataFrame

  Args:
    - row (pd.Series): row of a DataFrame

  Returns:
    - value[1] (np.float): next day price value
  """
  value = np.fromstring(row['Stock prices after (np.ndarray)'].strip("[]"), sep=' ') #.fromstring(dtype=float)
  return value[1]

df['Adj Close next day'] = df.apply(get_price_next_day, axis=1)

cleaned_columns = cleaned_columns + ['Adj Close next day']

print(df.head(1))

   Date  Unnamed: 0  Adj Close item 5 item 4 item 5.02 item_1 item_1a item 7  \
0    95  2019-03-05  76.549049    NaN    NaN       NaN    NaN     NaN    NaN   

  item 8.01  ... item 1.05 item 5.08  item 5.05 item 601 item 8.1 item 4.02  \
0       NaN  ...       NaN       NaN        NaN      NaN      NaN       NaN   

   item 2 item 4.01 item 3.03 Adj Close next day  
0     NaN       NaN       NaN          76.549049  

[1 rows x 165 columns]


## Arrange items

- **eightk_items** -> List: includes all columns with 8K text
- **tenk_text_items** -> List: includes all columns with 10K text (Management Discussion, Risk analysis)
- **tenq_text_items** -> List: includes all columns with 10Q text (Management Discussion, Risk analysis)
- **tenkq_numerical_items** -> List: includes all the numerical data found in the 10K/Q Filings
- **metadata_items** -> List: includes all columns with metadata (Labels etc.)
- **timeseries_items** -> List: includes all columns with timeseries data for the corresponding company

In [11]:
eightk_items = []
tenk_text_items = ['item_1', 'item_1a']
tenq_text_items = ['part_1_item_2', 'part_2_item_1a']
tenkq_numerical_items = []
metadata_items = ['Adj Close', 'Stock movement 1', 'Stock movement 7', 'Stock movement 30', 'Filing', 'Time', '10', 'Ticker', 'Date']
timeseries_items = ['Stock prices before (np.ndarray)', 'Stock prices after (np.ndarray)']

item_pattern = re.compile(r"^(Item\s[1-9][\.\d]*)", re.IGNORECASE)

for i in range(len(cleaned_columns)):
  match = item_pattern.match(cleaned_columns[i])
  if match:
    eightk_items.append(match.group(0))

print(eightk_items)
print(len(eightk_items))

other = eightk_items + tenk_text_items + tenq_text_items + metadata_items + timeseries_items

print("OTHER:",other)

for col in cleaned_columns:
  if col not in other:
    tenkq_numerical_items.append(col)

print("TENKQNUM:", tenkq_numerical_items)

df = df[cleaned_columns]

print(df.head(1))

['item 5', 'item 4', 'item 5.02', 'item 7', 'item 8.01', 'item 9.01', 'item 7.01', 'item 5.07', 'item 2.02', 'item 3.02', 'item 2.03', 'item 1.01', 'item 1.02', 'item 2.04', 'item 2.06', 'item\u202f9.01', 'item 3.01', 'item 3', 'item 5.04', 'item 5.03', 'item 9', 'item 2.05', 'item 8', 'item 2.01', 'item 5.01', 'item 1', 'item 6', 'item 12', 'item 1.04', 'item 1.05', 'item 5.08', 'item 5.05', 'item 601', 'item 8.1', 'item 4.02', 'item 2', 'item 4.01', 'item 3.03']
38
OTHER: ['item 5', 'item 4', 'item 5.02', 'item 7', 'item 8.01', 'item 9.01', 'item 7.01', 'item 5.07', 'item 2.02', 'item 3.02', 'item 2.03', 'item 1.01', 'item 1.02', 'item 2.04', 'item 2.06', 'item\u202f9.01', 'item 3.01', 'item 3', 'item 5.04', 'item 5.03', 'item 9', 'item 2.05', 'item 8', 'item 2.01', 'item 5.01', 'item 1', 'item 6', 'item 12', 'item 1.04', 'item 1.05', 'item 5.08', 'item 5.05', 'item 601', 'item 8.1', 'item 4.02', 'item 2', 'item 4.01', 'item 3.03', 'item_1', 'item_1a', 'part_1_item_2', 'part_2_item_1

### Text

Preprocessing Text columns

- Reference: https://doi.org/10.1016/j.dss.2022.113892

In [12]:
eightk_items
tenk_text_items
tenq_text_items
#label_columns = ['Adj Close', 'Stock movement 1', 'Stock movement 7', 'Stock movement 30', 'Adj Close next day', 'Filing', 'Time', '10', 'Ticker', 'Date']


text_columns = eightk_items + tenk_text_items + tenq_text_items + metadata_items

text_df = df[text_columns]

print(text_df.head(1))

  item 5 item 4 item 5.02 item 7 item 8.01 item 9.01  item 7.01 item 5.07  \
0    NaN    NaN       NaN    NaN       NaN       NaN        NaN       NaN   

   item 2.02  item 3.02  ...  \
0        NaN        NaN  ...   

                                      part_2_item_1a  Adj Close  \
0  ITEM 1A. RISK FACTORS\nRisks, Uncertainties an...  76.549049   

   Stock movement 1  Stock movement 7  Stock movement 30  Filing      Time  \
0         75.598679         77.384247            74.5989    True  16:08:06   

     10  Ticker  Date  
0  True       A    95  

[1 rows x 51 columns]


#### Get a complete text corpus

- input: Dataframe with only text columns

In [13]:
raw_path = "/content/text-preprocessing/text_corpus_generated.txt"

In [14]:
# Select text columns automatically:
# text_cols = text_df.select_dtypes(include=['object', 'string']).columns

text_df.fillna('').to_csv(raw_path, sep = '|', index=False, header=False)

#### Clean the text corpus partially

Goal:
- finding the rare words
- and common phrases

without to much noise

##### Replace abbreviations after: '
- t: to not
- s: append to the word coming before (either plural or is)
- ve: have
- d: would
- m: am
- ll: will
- re: are
- doesn't: does not
- can't: can not
- won't: will not
- don't: do not
- I've: i have
- I'd: i would
- I'm: i am
- I'll: i will
- She's: she is
- He's: he is
- It's: it is
- there's: there is
- they're: they are
- we're: we are
- you've: you have
- you're: you are
- couldn't: could not
- shouldn't: should not
- wouldn't: would not

In [15]:
def replace_abbreviations(text):
  # Define contraction patterns and their replacements
  contractions = {
    r"\bdoesn't\b": "does not",
    r"\bcan't\b": "can not",
    r"\bwon't\b": "will not",
    r"\bdon't\b": "do not",
    r"\bI've\b": "i have",
    r"\bI'd\b": "i would",
    r"\bI'm\b": "i am",
    r"\bI'll\b": "i will",
    r"\bShe's\b": "she is",
    r"\bHe's\b": "he is",
    r"\bIt's\b": "it is",
    r"\bthere's\b": "there is",
    r"\bthey're\b": "they are",
    r"\bwe're\b": "we are",
    r"\byou've\b": "you have",
    r"\byou're\b": "you are",
    r"\bcouldn't\b": "could not",
    r"\bshouldn't\b": "should not",
    r"\bwouldn't\b": "would not",
  }

  for pattern, replacement in contractions.items():
    text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)

  return text

##### Remove contact details

In [16]:
def remove_contact_details(text):
  # Regular expression patterns
  email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
  phone_pattern = r'(\+?\d{1,3}[-.\s]?)?(\(?\d{3}\)?[-.\s]?)?[\d.\s-]{7,}'
  url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
  social_media_pattern = r'@[a-zA-Z0-9_]+'

  # Substitute patterns with an empty string
  text = re.sub(email_pattern, '', text)          # Remove email addresses
  text = re.sub(phone_pattern, '', text)          # Remove phone numbers
  text = re.sub(url_pattern, '', text)            # Remove URLs
  text = re.sub(social_media_pattern, '', text)   # Remove social media handles

  # Remove any extra whitespace that might result from the removals
  text = re.sub(r'\s+', ' ', text).strip()

  return text

##### Remove stop words

- Reference: https://doi.org/10.1145/2783258.278338
- Cite: Bird, Steven, Edward Loper and Ewan Klein (2009), Natural Language Processing with Python. O’Reilly Media Inc.

In [17]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [18]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [19]:
def read_stop_word_list():
  """
  write List with stop words from file

  Returns:
    - stop_words (List[]): List with stop words
  """
  with open("/content/text-preprocessing/StopWords_Generic.txt", "r") as f:
    stop_words = [line.strip().lower() for line in f]

    return stop_words

In [20]:
def remove_stopwords_and_punctuation(text):
  """
  remove stop words and punctuation from a string

  Args:
    - text (string): text input

  Returns:
    - filtered_text (string): text without punctuation and stop words
  """

  # convert to tokens
  word_tokens = word_tokenize(text)
  stop_words_loughran = read_stop_word_list()

  filtered_text = [w for w in word_tokens if w not in stop_words_loughran]

  filtered_text = [w for w in word_tokens if w.isalpha()]

  return filtered_text

In [21]:
def remove_punctuation(text):
  """
  remove the punctuation from a string

  Args:
    - text (string): input string

  Returns:
    - filtered_text (string): string without punctuation, except for elements in sentence_punctuation
  """
  # keep sentence punctuation for Word2Vec methods
  sentence_punctuation = ['.', '!', '?', ':', ';', '\n']
  # convert to tokens
  word_tokens = word_tokenize(text)
  filtered_text = [w for w in word_tokens if w.isalpha() or w in sentence_punctuation]

  return filtered_text

##### append plural or genitive "s" to the word

In [22]:
def move_s_back(tokens):
  """
  standalone s' are appended to the previous word (due to removed punctuation)

  Args:
    - tokens (List[]): List of word tokens

  Returns:
    - result (List[]): List of word tokens with the s appended
  """
  result = []
  i = 0
  while i < len(tokens):
    if tokens[i] == "s" and i > 0:
      # Append "s" to the previous word in the result list
      result[-1] += "s"
    else:
      # Add the token to the result list as-is
      result.append(tokens[i])
    i += 1
  return result

##### Replace positive and negative numbers and dates

In [23]:
from dateutil import parser

In [24]:
def replace_nums_dates(text):
  """
  replace numbers and dates with a placeholder

  Args:
    - text (string): input string

  Returns:
    - text (string): string with replacements
  """
  number_pattern = r'\b\d{1,3}(?:,\d{3})*(?:\.\d+)?\b'
  date_pattern = r'\b(?:\d{1,2}[-/])?(?:\d{1,2}[-/])?\d{2,4}\b|\b\d{4}\b'

  text = re.sub(number_pattern, "NUMBER", text)
  text = re.sub(date_pattern, "DATE", text)

  return text

##### Create the partially cleaned text corpus

- tokenize

In [25]:
def create_corpus_cleaned(text, tokens):
  """
  calls preprocessing functions

  Args:
    - text (string): input string
    - tokens (boolean): set True or False depending on returntype

  Returns:
    - token_list (List[string]): List of tokens
    - result_text (string): text
  """
  text = remove_contact_details(text)
  text = replace_nums_dates(text)

  # lowercase
  text = text.lower()

  # ignores case
  text = replace_abbreviations(text)

  # Or remove_stopwords_and_punctuation, does not work, out of personal experience reduces performance for some models
  token_list = remove_punctuation(text)
  token_list = move_s_back(token_list)

  if tokens:
    return token_list
  else:
    result_text = " ".join(token_list)
    return result_text

In [26]:
def read_text_corpus(path):
  """
  read from file

  Args:
    - path (string): path to file

  Returns:
    - text (string): file content
  """
  with open(path, "r") as f:
    text = f.read()

  return text

In [27]:
text = read_text_corpus(path="/content/text-preprocessing/text_corpus_generated.txt")

clean_tokens = create_corpus_cleaned(text, tokens=True)

##### Prerequisits for further preprocessing

extract from the text corpus:

- common phrases
- rare words

In [28]:
from nltk import ngrams
from collections import Counter
import string

#### Find common phrases

possible improvement:
- calculate the distance between the phrases (levenstein, cos distance)
- group them, depending on the distance

In [29]:
def get_common_phrases(tokens, n=10, top_k=1000):
  """
  Returns top_k most common phrases

  Args:
    tokens (List[]): text tokens
    n (Int): ngram window size
    top_k (Int): number phrases to be returned

  Returns:
    common_phrases (List[Tuple(List, Int)])
  """
  # Generate n-grams (phrases of n words)
  n_grams = ngrams(tokens, n)
  n_gram_counts = Counter(n_grams)
  # Get the most common phrases
  common_phrases = n_gram_counts.most_common(top_k)

  return common_phrases

In [30]:
common_phrases_cleaned = get_common_phrases(clean_tokens, n=10, top_k=10)

# return a dtype: List[] with the most common phrases
common_phrases_list = []
for tu in common_phrases_cleaned:
  tokenized = list(tu[0])
  common_phrases_list.append(tokenized)

In [31]:
print(common_phrases_cleaned[:2])
print(common_phrases_list[:2])

[(('for', 'the', 'three', 'and', 'six', 'months', 'ended', 'april', 'number', 'date'), 47), (('respectively', 'when', 'compared', 'to', 'the', 'same', 'periods', 'last', 'year', '.'), 26)]
[['for', 'the', 'three', 'and', 'six', 'months', 'ended', 'april', 'number', 'date'], ['respectively', 'when', 'compared', 'to', 'the', 'same', 'periods', 'last', 'year', '.']]


##### Find words

- rare
- most common

In [32]:
def get_rare_words(tokens, frequency_threshold=0.01):
  """
  get rare words based on a frequency threshold

  Args:
    - tokens (List[string]): List of tokens
    - frequency_threshold (float): selection of words based on occurrence count in %

  Returns:
    - bottom_words (List[List[string]]): List of rare words
    - top_words (List[List[string]]): List of most frequent word(s)
  """
  # Count word frequency
  word_counts = Counter(tokens)
  # Sort words by frequency (ascending)
  sorted_counts = sorted(word_counts.values())
  # Threshold frequency
  percent_index_bottom = int(len(sorted_counts) * frequency_threshold)
  percent_index_top = int(len(sorted_counts) * 1-frequency_threshold)

  if percent_index_bottom < 1:
    percent_index_bottom = 1
  bottom_percent_words = sorted_counts[percent_index_bottom -1]

  if percent_index_top < 1:
    percent_index_top = -1 #end of list (ascending)
  top_percent_words = sorted_counts[percent_index_top]

  # Get all words with frequencies at or below
  bottom_words = [[token] for token, count in word_counts.items() if count <= bottom_percent_words]
  # Get all words with frequencies at or above
  top_words = [[token] for token, count in word_counts.items() if count >= top_percent_words]

  return bottom_words, top_words

In [33]:
bottom_words_cleaned, top_words_cleaned = get_rare_words(clean_tokens, frequency_threshold=0.01)

print(top_words_cleaned)
print(bottom_words_cleaned[-10:])

[['the']]
[['typical'], ['methodology'], ['climate'], ['stricter'], ['adaptations'], ['noteholders'], ['severity'], ['frequency'], ['extreme'], ['weather']]


##### Remove words and phrases

In [34]:
to_be_removed = common_phrases_list + bottom_words_cleaned

print(to_be_removed[:2])

[['for', 'the', 'three', 'and', 'six', 'months', 'ended', 'april', 'number', 'date'], ['respectively', 'when', 'compared', 'to', 'the', 'same', 'periods', 'last', 'year', '.']]


In [35]:
def exclude_phrases_tokens(tokens, phrases):
    """
    Removes tokens that belong to any phrase in `phrases`, including overlapping ones.
    Each phrase is a list of tokens.

    Args:
      - tokens (List[string]): tokens to remove from
      - phrases (List[List[string]]): List of strings to remove from "tokens"
    """
    phrases = [tuple(p) for p in phrases]
    max_len = max(len(p) for p in phrases)

    n = len(tokens)
    remove_indices = set()

    for i in range(n):
        for L in range(1, max_len + 1):
            if i + L <= n and tuple(tokens[i:i+L]) in phrases:
                remove_indices.update(range(i, i + L))

    # Keep only tokens not marked for removal
    return [tok for idx, tok in enumerate(tokens) if idx not in remove_indices]

## Cleaned Text

- save

In [36]:
def write_corpus(text, path):
  """
  write text to file

  Args:
    - text (string): text to be written to file
    - path (string): path to file
  """
  with open(path, "w") as f:
    f.write(text)

In [37]:
cleaned_path = "/content/text-preprocessing/text_corpus_clean_generated.txt"

In [38]:
partially_clean_text = create_corpus_cleaned(text, tokens=False)

print(partially_clean_text[:100])

cleaned_tokens = exclude_phrases_tokens(clean_tokens, to_be_removed)

cleaned_text = " ".join(cleaned_tokens)

print("Cleaned Text:", cleaned_text[:100])

item number . managements discussion and analysis of financial condition and results of operations u
Cleaned Text: item number . managements discussion and analysis of financial condition and results of operations u


In [39]:
write_corpus(cleaned_text, cleaned_path)

## Cleaned Text inside DataFrame

- to keep the context of the text
- takes a long time
- Idea: unique separator when extracting the text cells, and return them back into the DataFrame based on position and Nr. separator count

In [40]:
def preprocess_text(text, to_be_removed):
  """
  executes preprocessing methods to clean a string

  Args:
    - text (string): text to preprocess
    - to_be_removed (List[List[string]]): phrases and words to remove

  Returns:
    - result_text (string): output string
  """
  text = remove_contact_details(text)
  text = replace_nums_dates(text)

  # lowercase:
  text = text.lower()

  # ignores case
  text = replace_abbreviations(text)

  # Or remove_stopwords_and_punctuation, does not work, out of personal experience reduces performance for some models
  token_list = remove_punctuation(text)
  token_list = move_s_back(token_list)
  token_list = exclude_phrases_tokens(token_list, to_be_removed)

  result_text = " ".join(token_list)

  return result_text

In [41]:
def get_text(row, to_be_removed):
  """
  Loops over each string column of a DataFrame row and executes a string preprocessing method

  Args:
    - row (pd.Series): row inside a Dataframe
    - to_be_removed (List[List[string]]): List of phrases and words to remove

  Returns:
    - row (pd.Series): processed pd.Series
  """
  columns = list(row.index)
  for col in columns:
    if pd.notna(row[col]) and col not in metadata_items:
      row[col] = preprocess_text(row[col], to_be_removed)

  return row

In [42]:
# Apply a function on every row of a pd.DataFrame
processed_text_df = text_df.apply(lambda row: get_text(row, to_be_removed), axis=1)

processed_text_df.to_csv("/content/text-preprocessing/processed_text_df_generated.csv")

print(processed_text_df.head(1))

   item 5  item 4  item 5.02  item 7  item 8.01 item 9.01  item 7.01  \
0     NaN     NaN        NaN     NaN        NaN       NaN        NaN   

  item 5.07  item 2.02  item 3.02  ...  \
0       NaN        NaN        NaN  ...   

                                      part_2_item_1a  Adj Close  \
0  item . risk factors risks uncertainties and ot...  76.549049   

   Stock movement 1  Stock movement 7  Stock movement 30  Filing      Time  \
0         75.598679         77.384247            74.5989    True  16:08:06   

     10  Ticker  Date  
0  True       A    95  

[1 rows x 51 columns]


## The cleaned text corpus can now be used for training

- make it machine readable
  - word2Vec (Sentence delimiters were kept)
  - etc.

## The cleaned DataFrame can be used for further processing